# A joint model CAR (model 1) 
## Two independent processes
 $$ [P , S ] = [P ] [ S ] $$

In [1]:
## Import code:
setwd('/apps/external_plugins/biospytial_rwrapper/biospytial.rwrapper/R/')

In [2]:
rm(list=ls())                                                                                     
source("init_data.R")                                                                             
# load the building function                                                                      
source("joint.binomial.bymCARModel1.R")
## Fix the burnin and sample to be consistent with the other models
n.sample = 10000                                                                                  
burnin=10000                                                                                      
postburnin = burnin +1000                                                                         
thin = 1                                                                                          
verbose = TRUE                           
results  <- joint.binomial.bymCARModel1(formula_S = formula_sample, 
                                        formula_P = formula_presence,
                                        n.sample=n.sample,
                                        data = DataFrame,
                                        burnin=burnin,
                                        postburnin=postburnin,
                                        thin=thin,
                                        verbose=TRUE) 


Loading required package: MASS
Loading required package: Rcpp

Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Setting up the model.
Performing burnin period for Sample Effort Model 
  |======================================================================| 100%
Summarising results.
Finished in  87.5 seconds.
Setting up the model.
Performing burnin period for Presence model 
  |======================================================================| 100%
Summarising results.
Finished in  84.7 seconds.
Sampling from the joint model 10000 post burnin and thinned (if requested) samples.
  |======================================================================| 100%
[1] "Compiling summary for S process"
[1] "Compiling summary for the P process"


In [154]:
Ysamp = model.frame(formula = formula_sample,data =DataFrame )[1]
Ypres = model.frame(formula = formula_presence,data =DataFrame )[1]
names(Ysamp) <- 'obs'
names(Ypres) <- 'obs'
#Y = rbind(Ysamp,Ypres)
##
Y = rbind(Ypres,Ysamp)

In [155]:
#mean.prob = rbind(results$S$mean.prob,results$P$mean.prob)
## Let´s try to invert the order 
mean.prob = rbind(results$P$mean.prob,results$S$mean.prob)


In [160]:
dtrials = rep(1,2*n)
dtrials = rep(1,n)

In [166]:
Deviance <- function(obs,mean.prob,trials) {
    devfit = -2 * sum(dbinom(x=obs,size=trials,prob=mean.prob,log=TRUE),na.rm=TRUE)
    return(devfit)
}


In [170]:
devfitpres = Deviance(Ypres$obs,results$P$mean.prob,dtrials)

In [ ]:
devfit

In [157]:
devfit

[1] 8267.876

In [153]:
devfit

[1] 22171.69

In [119]:
S.loglike = results$S$samples$loglike
P.loglike = results$P$samples$loglike

In [120]:
## Inverting the loglikelihood does not have an impact
nn = cbind(S.loglike,P.loglike)
#nn = cbind(P.loglike,S.loglike)

In [121]:
## Lets analyse why the dic changes so much
loglike = nn
mean_dev = sum(loglike,na.rm=TRUE) / nrow(loglike)
pd = mean_dev  - devfit
DIC = devfit + 2 * pd

In [122]:
DIC

[1] -24744.6

In [123]:
common.modelfit(nn,devfit)

DIC           p.d          WAIC           p.w          LMPL 
  -17025.8616   -19598.7759     3558.6069      765.3822    -1930.0073 
loglikelihood 
  -11085.8452

In [88]:
## obs inverted mean.prob inverted
common.modelfit(nn,devfit)

DIC           p.d          WAIC           p.w          LMPL 
   -3122.0475    -5694.9618     3558.6069      765.3822    -1930.0073 
loglikelihood 
   -4133.9381

In [75]:
## obs inverted no mean.prob 
common.modelfit(nn,devfit)

DIC           p.d          WAIC           p.w          LMPL 
  -59180.9690   -61753.8834     3558.6069      765.3822    -1930.0073 
loglikelihood 
  -32163.3989

In [13]:
results$P$modelfit

DIC           p.d          WAIC           p.w          LMPL 
    1343.4549      489.0117     1375.3325      382.2678     -815.1783 
loglikelihood 
    -182.7157

## Check the fitted dev for a single S or P and check that the values are the same

> Good! TEst passed!

In [16]:
mean.prob = results$P$mean.prob
dtrials = rep(1,n)

devfit = -2 * sum(dbinom(x=Ypres$obs,size=dtrials,prob=mean.prob,log=TRUE),na.rm=TRUE)

common.modelfit(P.loglike,devfit)


DIC           p.d          WAIC           p.w          LMPL 
    1343.4549      489.0117     1375.3325      382.2678     -815.1783 
loglikelihood 
    -182.7157

In [27]:
svar = apply(S.loglike,2,var)

In [3]:
alldev = results$S$deviance.fitted + results$P$

[1] 1268.906

In [12]:
results$S$summary.results

,Median,2.5%,97.5%,n.sample,% accept,n.effective,Geweke.diag
(Intercept),0.1312,-1.0496,1.6348,10000,53.7,6.9,0.5
Disttoroadm,-0.0002,-0.0004,-0.0001,10000,53.7,1503.7,-0.3
Populationm,0.0000,-0.0001,0.0001,10000,53.7,1724.2,-2.6
tau2,5.2243,3.2542,8.7656,10000,100.0,13.6,1.4
sigma2,0.0754,0.0169,0.4669,10000,100.0,2.7,-2.5


In [4]:
## Check that M_bis[1] and nrow(TDF) have the same  value

## Extract deviance.fit stack the sample logs and deviance fit and use common.modelfit

In [5]:
sum(results$S$samples$loglike,na.rm=TRUE)

[1] -8541350

In [ ]:
model

In [37]:
str(nn)

 num [1:10000, 1:8120] NA NA NA NA NA NA NA NA NA NA ...


In [35]:
nvar = apply(nn,2,var)

In [36]:
length(nvar)

[1] 8120

In [30]:
length(svar)

[1] 4060

In [22]:
common.modelfit()

In [ ]:
post.sample_p = data.frame(results$S$samples$fitted)
post.presence_p = data.frame(results$P$samples$fitted)

In [ ]:
pss  <- post.sample_p %>% mutate_all(function(p) rbernoulli(1,p))
pps <- post.presence_p %>% mutate_all(function(p) rbernoulli(1,p))                                        

In [ ]:

sumpps <- colSums(pps)

In [ ]:
sumpss <- colSums(pss)

In [ ]:
prob_pps <- sumpps / length(pps)

In [ ]:
prob_pss <- sumpss / length(pss)

In [ ]:
probpres = prob_pps * prob_pss

In [12]:
## save to posterity
# Save output sample to display in python
## Save the complete chain trace for analysing in Python,
TDF = read.csv("/outputs/training_data_sample_puebla_p9_abies_pinophyta.csv")
## Order it according to the id of the cell
## This is important because the adjancy matrix rows need to be the same
TDF = TDF[order(TDF$cellids),]

In [13]:
chain = cbind(TDF$cell_ids,probpres,prob_pps,prob_pss)
file_ = '/outputs/presence_only_models/modelCAR_model1_fitted.csv'
write.csv(chain,file_)